Import library

In [9]:
from CoffeeLibrary.PreProcess import *
from CoffeeLibrary.FeatureCreate import *
from CoffeeLibrary.TrainTest import *
from sklearn.metrics import mean_squared_error

Load and process data

In [10]:
path = 'coffee_df_with_type_and_region.csv'

df = pd.read_csv(path, sep = ',', thousands=',')

prep = PreProcess(df)

prep.clean_origin('origin')
prep.clean_location('location')
prep.standardize_price('est_price')
prep.standardize_agtron('agtron')
prep.drop_cols(['slug', 'all_text', 'with_milk', 'name', 'review_date'])
prep.drop_nas()

df = prep.df 

/Users/ericfrey/opt/anaconda3/envs/coffee/lib/python3.10/site-packages/dask/dataframe/core.py:6751: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


  0%|          | 0/8 [00:00<?, ?it/s]

Country Cleaning Report:
	2022 values cleaned (88.61%)
	185 values unable to be parsed (8.11%), set to NaN
Result contains 2097 (91.89%) values in the correct format and 185 null values (8.11%)


/Users/ericfrey/Documents/computing/coffee/CoffeeLibrary/PreProcess.py:38: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  self.df[['amount', 'units']] = self.df['est_price'].apply(lambda row: str(row).split('/')[-1].replace(',','')).str.split(' ', 1, expand = True)


Create Features

In [3]:
prep.df = pd.concat([prep.df, FeatureCreate(prep.df, 'desc_1').top_n_unigram(20)], axis=1)
prep.df = pd.concat([prep.df, FeatureCreate(prep.df, 'desc_2').top_n_unigram(20)], axis=1)
prep.df = pd.concat([prep.df, FeatureCreate(prep.df, 'desc_3').top_n_unigram(20)], axis=1)
prep.df = pd.concat([prep.df, FeatureCreate(prep.df, 'desc_1').top_n_bigram(20)], axis=1)
prep.df = pd.concat([prep.df, FeatureCreate(prep.df, 'desc_3').top_n_bigram(20)], axis=1)
prep.df.drop(['desc_1','desc_2','desc_3'], axis=1, inplace=True)

df = pd.get_dummies(prep.df, columns=['origin','location','roast','roaster'])

Split Dataframe

In [4]:
features = df.columns[df.columns != 'rating']
X_train, X_test, y_train, y_test = split(df, features)

Train Model and Predict

In [5]:
regressor = train_model(X_train, y_train, features)
results = predict(X_test, y_test, regressor, features)

/Users/ericfrey/Documents/computing/coffee/CoffeeLibrary/TrainTest.py:30: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X, y)


Compute mean squared error

In [6]:
mean_squared_error(results.Actual, results.Predicted)

0.037869980119284295